## **Mfano: kurekebisha Phi-4-mini-reasoning kwa kutumia Apple MLX Framework**

Hii inatokana na data kutoka FreedomIntelligence/medical-o1-reasoning-SFT, ambayo inaboresha uwezo wa Phi-4-mini-reasoning kufikiria kuhusu matukio ya kitabibu. Ikiwa unataka kujifunza zaidi kuhusu data hii, unaweza kutumia kiungo hiki kujifunza zaidi: https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT 

*Note: Tafadhali tumia vifaa vya Apple silicon vyenye kumbukumbu ya 64gb kuendesha mfano huu*


In [ ]:
! pip install -U mlx-lm

In [1]:
! pip install datasets

### **Data**

Tunahitaji kugawanya data katika train na valid na kuiweka kwenye folda ya data. Kumbuka kwamba apple mlx huipa majina data ya mafunzo kwa chaguo-msingi. train inaitwa train.jsonl na valid inaitwa valid.json.


In [2]:
from datasets import load_dataset

/Users/lokinfey/miniforge3/envs/mlxdev/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
prompt_template = """<|user|>{}<|end|><|assistant|><think>{}</think>{}<|end|>"""


def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # text = prompt_template.format(input, cot, output) + "<|end|>"
        text = prompt_template.format(input, cot, output) + "<|endoftext|>"
        texts.append(text)
    return {
        "text": texts,
    }


In [4]:
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en", trust_remote_code=True)
split_dataset = dataset["train"].train_test_split(test_size=0.2, seed=200)


In [5]:
train_dataset = split_dataset['train']
validation_dataset = split_dataset['test']

In [6]:
train_dataset = train_dataset.map(formatting_prompts_func, batched = True,remove_columns=["Question", "Complex_CoT", "Response"])
train_dataset.to_json("./data/train.jsonl")

Creating json from Arrow format: 100%|██████████| 16/16 [00:00<00:00, 93.83ba/s]


46545696

In [7]:

validation_dataset = validation_dataset.map(formatting_prompts_func, batched = True,remove_columns=["Question", "Complex_CoT", "Response"])
validation_dataset.to_json("./data/valid.jsonl")

Creating json from Arrow format: 100%|██████████| 4/4 [00:00<00:00, 95.53ba/s]


11674511

In [8]:
! python -m mlx_lm.lora --model ./phi-4-mini-reasoning --train --data ./data  --iters 100

Calling `python -m mlx_lm.lora...` directly is deprecated. Use `mlx_lm.lora...` or `python -m mlx_lm lora ...` instead.
Loading pretrained model
Loading datasets
Training
Trainable parameters: 0.027% (1.049M/3836.022M)
Starting training..., iters: 100
Iter 1: Val loss 2.827, Val took 89.684s
Iter 10: Train loss 2.761, Learning Rate 1.000e-05, It/sec 0.174, Tokens/sec 396.129, Trained Tokens 22765, Peak mem 23.744 GB
Iter 20: Train loss 2.110, Learning Rate 1.000e-05, It/sec 0.152, Tokens/sec 383.076, Trained Tokens 47982, Peak mem 28.763 GB
Iter 30: Train loss 1.931, Learning Rate 1.000e-05, It/sec 0.180, Tokens/sec 412.540, Trained Tokens 70944, Peak mem 28.763 GB
Iter 40: Train loss 1.925, Learning Rate 1.000e-05, It/sec 0.180, Tokens/sec 431.207, Trained Tokens 94878, Peak mem 28.763 GB
Iter 50: Train loss 1.804, Learning Rate 1.000e-05, It/sec 0.174, Tokens/sec 413.008, Trained Tokens 118679, Peak mem 28.763 GB
Iter 60: Train loss 1.710, Learning Rate 1.000e-05, It/sec 0.169, Token

In [9]:
! python -m mlx_lm.generate --model ./phi-4-mini-reasoning --adapter-path ./adapters --max-token 4096 --prompt "A 54-year-old construction worker with a long history of smoking presents with swelling in his upper extremity and face, along with dilated veins in this region. After conducting a CT scan and venogram of the neck, what is the most likely diagnosis for the cause of these symptoms?" --extra-eos-token "<|end|>"

<frozen runpy>:128: RuntimeWarning: 'mlx_lm.generate' found in sys.modules after import of package 'mlx_lm', but prior to execution of 'mlx_lm.generate'; this may result in unpredictable behaviour
Calling `python -m mlx_lm.generate...` directly is deprecated. Use `mlx_lm.generate...` or `python -m mlx_lm generate ...` instead.
<think>Okay, let's see. The patient is a 54-year-old construction worker with a long history of smoking. He has swelling in his upper extremity and face, and dilated veins in this region. The CT scan and venogram of the neck are done. Hmm, what could be causing these symptoms?

First, let's think about the symptoms. Swelling in the upper extremity and face, along with dilated veins. That sounds like edema, which is swelling due to fluid accumulation. The dilated veins might be related to increased pressure or venous insufficiency. 

Now, considering the patient's history of smoking, that might be a clue. Smoking can contribute to vascular issues, like atheroscler


---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya kutafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
